<center>
    <img src="https://gitlab.com/ibm/skills-network/courses/placeholder101/-/raw/master/labs/module%201/images/IDSNlogo.png"  width="300" alt="cognitiveclass.ai logo"  />
</center>

<h1 align=center><font size = 5>Assignment: SQL Notebook for Peer Assignment</font></h1>

Estimated time needed: **60** minutes.

## Introduction

Using this Python notebook you will:

1.  Understand the Spacex DataSet
2.  Load the dataset  into the corresponding table in a Db2 database
3.  Execute SQL queries to answer assignment questions


## Overview of the DataSet

SpaceX has gained worldwide attention for a series of historic milestones.

It is the only private company ever to return a spacecraft from low-earth orbit, which it first accomplished in December 2010.
SpaceX advertises Falcon 9 rocket launches on its website with a cost of 62 million dollars wheras other providers cost upward of 165 million dollars each, much of the savings is because Space X can reuse the first stage.

Therefore if we can determine if the first stage will land, we can determine the cost of a launch.

This information can be used if an alternate company wants to bid against SpaceX for a rocket launch.

This dataset includes a record for each payload carried during a SpaceX mission into outer space.


### Download the datasets

This assignment requires you to load the spacex dataset.

In many cases the dataset to be analyzed is available as a .CSV (comma separated values) file, perhaps on the internet. Click on the link below to download and save the dataset (.CSV file):

<a href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2021-01-01" target="_blank">Spacex DataSet</a>


### Store the dataset in database table

**it is highly recommended to manually load the table using the database console LOAD tool in DB2**.

<img src = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/images/spacexload.png">

Now open the Db2 console, open the LOAD tool, Select / Drag the .CSV file for the  dataset, Next create a New Table, and then follow the steps on-screen instructions to load the data. Name the new table as follows:

**SPACEXDATASET**

**Follow these steps while using old DB2 UI which is having Open Console Screen**

**Note:While loading Spacex dataset, ensure that detect datatypes is disabled. Later click on the pencil icon(edit option).**

1.  Change the Date Format by manually typing DD-MM-YYYY and timestamp format as DD-MM-YYYY HH\:MM:SS.

    Here you should place the cursor at Date field and manually type as DD-MM-YYYY.

2.  Change the PAYLOAD_MASS\_\_KG\_  datatype  to INTEGER.

<img src = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/images/spacexload2.png">


**Changes to be considered when having DB2 instance with the new UI having Go to UI screen**

*   Refer to this insruction in this <a href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Labs_Coursera_V5/labs/Lab%20-%20Sign%20up%20for%20IBM%20Cloud%20-%20Create%20Db2%20service%20instance%20-%20Get%20started%20with%20the%20Db2%20console/instructional-labs.md.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2021-01-01">link</a> for viewing  the new  Go to UI screen.

*   Later click on **Data link(below SQL)**  in the Go to UI screen  and click on **Load Data** tab.

*   Later browse for the downloaded spacex file.

<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/images/browsefile.png" width="800"/>

*   Once done select the schema andload the file.

 <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/images/spacexload3.png" width="800"/>


In [1]:
# !pip install sqlalchemy==1.3.9
# !pip install ibm_db_sa
# !pip install ipython-sql

     |████████████████████████████████| 6.0 MB 22.5 MB/s eta 0:00:01
  Created wheel for sqlalchemy: filename=SQLAlchemy-1.3.9-cp38-cp38-linux_x86_64.whl size=1209497 sha256=033e68de7f15a257981561099893439ae1f622b0ebb581a69d994c7a7f9e2a4c
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/cb/43/46/fa638f2422554332b7865d600275b24568bf60e76104a94bb4
Successfully built sqlalchemy
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 1.4.22
    Uninstalling SQLAlchemy-1.4.22:
      Successfully uninstalled SQLAlchemy-1.4.22
     |████████████████████████████████| 42 kB 1.5 MB/s  eta 0:00:01
  Created wheel for prettytable: filename=prettytable-0.7.2-py3-none-any.whl size=13700 sha256=a5b7b802faca414434463695ce2073f6c60ce617db6dee1be4dce27b8cda08cc
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/48/6d/77/9517cb933af254f51a446f1a5ec9c2be3e45f17384940bce68
Successfully built prettytable


In [1]:
import ibm_db_sa
import ibm_db
import ibm_db_dbi
import pandas as pd

### Connect to the database

Using ibm_db API

In [2]:
#Replace the placeholder values with your actual Db2 hostname, username, and password:
dsn_hostname = "764264db-9824-4b7c-82df-40d1b13897c2.bs2io90l08kqb1od8lcg.databases.appdomain.cloud"
dsn_uid = "sbq39922"
dsn_pwd = "GZnFKjRPRI2lHHjc"

dsn_driver = "{IBM DB2 ODBC DRIVER}"
dsn_database = "bludb"
dsn_port = "32536"
dsn_protocol = "TCPIP"            # i.e. "TCPIP"
dsn_security = "SSL"              #i.e. "SSL"

In [3]:
#DO NOT MODIFY THIS CELL. Just RUN it with Shift + Enter
#Create the dsn connection string
dsn = (
    "DRIVER={0};"
    "DATABASE={1};"
    "HOSTNAME={2};"
    "PORT={3};"
    "PROTOCOL={4};"
    "UID={5};"
    "PWD={6};"
    "SECURITY={7};").format(dsn_driver, dsn_database, dsn_hostname, dsn_port, dsn_protocol, dsn_uid, dsn_pwd,dsn_security)

#print the connection string to check correct values are specified
print(dsn)

DRIVER={IBM DB2 ODBC DRIVER};DATABASE=bludb;HOSTNAME=764264db-9824-4b7c-82df-40d1b13897c2.bs2io90l08kqb1od8lcg.databases.appdomain.cloud;PORT=32536;PROTOCOL=TCPIP;UID=sbq39922;PWD=GZnFKjRPRI2lHHjc;SECURITY=SSL;


In [4]:
#DO NOT MODIFY THIS CELL. Just RUN it with Shift + Enter
#Create database connection

try:
    conn = ibm_db.connect(dsn, "", "")
    print ("Connected to database: ", dsn_database, "as user: ", dsn_uid, "on host: ", dsn_hostname)

except:
    print ("Unable to connect: ", ibm_db.conn_errormsg() )


Connected to database:  bludb as user:  sbq39922 on host:  764264db-9824-4b7c-82df-40d1b13897c2.bs2io90l08kqb1od8lcg.databases.appdomain.cloud


In [5]:
#Retrieve Metadata for the Database Server
server = ibm_db.server_info(conn)

print ("DBMS_NAME: ", server.DBMS_NAME)
print ("DBMS_VER:  ", server.DBMS_VER)
print ("DB_NAME:   ", server.DB_NAME)

DBMS_NAME:  DB2/LINUXX8664
DBMS_VER:   11.05.0600
DB_NAME:    BLUDB


In [6]:
#Retrieve Metadata for the Database Client / Driver
client = ibm_db.client_info(conn)

print ("DRIVER_NAME:          ", client.DRIVER_NAME) 
print ("DRIVER_VER:           ", client.DRIVER_VER)
print ("DATA_SOURCE_NAME:     ", client.DATA_SOURCE_NAME)
print ("DRIVER_ODBC_VER:      ", client.DRIVER_ODBC_VER)
print ("ODBC_VER:             ", client.ODBC_VER)
print ("ODBC_SQL_CONFORMANCE: ", client.ODBC_SQL_CONFORMANCE)
print ("APPL_CODEPAGE:        ", client.APPL_CODEPAGE)
print ("CONN_CODEPAGE:        ", client.CONN_CODEPAGE)

DRIVER_NAME:           libdb2.a
DRIVER_VER:            11.05.0500
DATA_SOURCE_NAME:      BLUDB
DRIVER_ODBC_VER:       03.51
ODBC_VER:              03.01.0000
ODBC_SQL_CONFORMANCE:  EXTENDED
APPL_CODEPAGE:         1208
CONN_CODEPAGE:         1208


### Connect to the database

Let us first load the SQL extension and establish a connection with the database


In [9]:
# %load_ext sql

**DB2 magic in case of old UI service credentials.**

In the next cell enter your db2 connection string. Recall you created Service Credentials for your Db2 instance before. From the **uri** field of your Db2 service credentials copy everything after db2:// (except the double quote at the end) and paste it in the cell below after ibm_db_sa://

<img src ="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_edX/images/URI.jpg">

in the following format

**%sql ibm_db_sa://my-username:my-password\@my-hostname:my-port/my-db-name**

**DB2 magic in case of new UI service credentials.**

<img src ="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/images/servicecredentials.png" width=600>  

*   Use the following format.

*   Add security=SSL at the end

**%sql ibm_db_sa://my-username:my-password\@my-hostname:my-port/my-db-name?security=SSL**


In [10]:
# # Enter the connection string for your Db2 on Cloud database instance below
# # %sql ibm_db_sa://my-username:my-password@my-hostname:my-port/my-db-name?security=SSL
# %sql ibm_db_sa://sbq39922:GZnFKjRPRI2lHHjc@764264db-9824-4b7c-82df-40d1b13897c2.bs2io90l08kqb1od8lcg.databases.appdomain.cloud:32536/bludb?security=SSL

In [11]:
#connection for pandas
pconn = ibm_db_dbi.Connection(conn)

In [12]:
#query statement to retrieve all rows in INSTRUCTOR table
query = "select TABSCHEMA, TABNAME, CREATE_TIME from SYSCAT.TABLES \
         where TABSCHEMA not in ('SYSIBM', 'SYSCAT', 'SYSSTAT', 'SYSIBMADM', 'SYSTOOLS', 'SYSPUBLIC')"

#retrieve the query results into a pandas dataframe
df = pd.read_sql(query, pconn)
df

,TABSCHEMA,TABNAME,CREATE_TIME
0,SBQ39922,PETSALE,2021-09-30 16:57:32.974730
1,SBQ39922,EMPLOYEES,2021-09-30 17:16:34.506400
2,SBQ39922,JOB_HISTORY,2021-09-30 17:16:34.616621
3,SBQ39922,JOBS,2021-09-30 17:16:34.742614
4,SBQ39922,DEPARTMENTS,2021-09-30 17:16:34.867631
5,SBQ39922,LOCATIONS,2021-09-30 17:16:34.992048
6,SBQ39922,PETRESCUE,2021-10-08 14:30:16.492096
7,SBQ39922,INSTRUCTOR,2021-10-08 20:53:19.796647
8,SBQ39922,INTERNATIONAL_STUDENT_TEST_SCORES,2021-10-08 21:20:34.296727
9,SBQ39922,CHICAGO_SOCIOECONOMIC_DATA,2021-10-09 08:47:35.352945


In [14]:
# retrieve columns in SPACEXDATASET
query = "select COLNAME, TYPENAME, LENGTH from SYSCAT.COLUMNS where TABNAME = 'SPACEXDATASET'"

#Execute the statement
selectStmt = ibm_db.exec_immediate(conn, query)

# #Fetch the Dictionary (for the first row only) - replace ... with your code
# ibm_db.fetch_both(selectStmt)

#Fetch all the rows
while ibm_db.fetch_row(selectStmt) != False:
   print (" COLNAME:",  ibm_db.result(selectStmt, "COLNAME"), " TYPENAME:",  ibm_db.result(selectStmt, "TYPENAME"), " LENGTH:",  ibm_db.result(selectStmt, "LENGTH"))

 COLNAME: DATE  TYPENAME: DATE  LENGTH: 4
 COLNAME: Time (UTC)  TYPENAME: TIME  LENGTH: 3
 COLNAME: BOOSTER_VERSION  TYPENAME: VARCHAR  LENGTH: 14
 COLNAME: LAUNCH_SITE  TYPENAME: VARCHAR  LENGTH: 12
 COLNAME: PAYLOAD  TYPENAME: VARCHAR  LENGTH: 61
 COLNAME: PAYLOAD_MASS__KG_  TYPENAME: INTEGER  LENGTH: 4
 COLNAME: ORBIT  TYPENAME: VARCHAR  LENGTH: 11
 COLNAME: CUSTOMER  TYPENAME: VARCHAR  LENGTH: 57
 COLNAME: MISSION_OUTCOME  TYPENAME: VARCHAR  LENGTH: 32
 COLNAME: Landing _Outcome  TYPENAME: VARCHAR  LENGTH: 22


In [13]:
#query statement to retrieve all rows in INSTRUCTOR table
query = "select COLNAME, TYPENAME, LENGTH from SYSCAT.COLUMNS where TABNAME = 'SPACEXDATASET'"

#retrieve the query results into a pandas dataframe
df = pd.read_sql(query, pconn)
df

,COLNAME,TYPENAME,LENGTH
0,DATE,DATE,4
1,Time (UTC),TIME,3
2,BOOSTER_VERSION,VARCHAR,14
3,LAUNCH_SITE,VARCHAR,12
4,PAYLOAD,VARCHAR,61
5,PAYLOAD_MASS__KG_,INTEGER,4
6,ORBIT,VARCHAR,11
7,CUSTOMER,VARCHAR,57
8,MISSION_OUTCOME,VARCHAR,32
9,Landing _Outcome,VARCHAR,22


In [15]:
query = "select count(*) from spacexdataset"

df = pd.read_sql(query, pconn)
df

,1
0,101


In [16]:
query = "select * from spacexdataset limit 10"

df = pd.read_sql(query, pconn)
df

,DATE,Time (UTC),BOOSTER_VERSION,LAUNCH_SITE,PAYLOAD,PAYLOAD_MASS__KG_,ORBIT,CUSTOMER,MISSION_OUTCOME,Landing _Outcome
0,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)
1,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of...",0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
2,2012-05-22,07:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525,LEO (ISS),NASA (COTS),Success,No attempt
3,2012-10-08,00:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),Success,No attempt
4,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),Success,No attempt
5,2013-09-29,16:00:00,F9 v1.1 B1003,VAFB SLC-4E,CASSIOPE,500,Polar LEO,MDA,Success,Uncontrolled (ocean)
6,2013-12-03,22:41:00,F9 v1.1,CCAFS LC-40,SES-8,3170,GTO,SES,Success,No attempt
7,2014-01-06,22:06:00,F9 v1.1,CCAFS LC-40,Thaicom 6,3325,GTO,Thaicom,Success,No attempt
8,2014-04-18,19:25:00,F9 v1.1,CCAFS LC-40,SpaceX CRS-3,2296,LEO (ISS),NASA (CRS),Success,Controlled (ocean)
9,2014-07-14,15:15:00,F9 v1.1,CCAFS LC-40,OG2 Mission 1 6 Orbcomm-OG2 satellites,1316,LEO,Orbcomm,Success,Controlled (ocean)


### Task 3

##### Display the total payload mass carried by boosters launched by NASA (CRS)


In [32]:
query = "select sum(payload_mass__kg_) as total_payload_mass__kg_ \
         from spacexdataset \
         where customer = 'NASA (CRS)'"

df = pd.read_sql(query, pconn)
df

,TOTAL_PAYLOAD_MASS__KG_
0,45596


In [36]:
query = "select customer, count(*) as launch_count \
         from spacexdataset \
         group by customer \
         order by launch_count desc \
         limit 3"

df = pd.read_sql(query, pconn)
df

,CUSTOMER,LAUNCH_COUNT
0,NASA (CRS),20
1,SpaceX,13
2,Iridium Communications,7


In [31]:
query = "select customer, sum(payload_mass__kg_) as total_payload_mass__kg_ \
         from spacexdataset \
         group by customer \
         order by total_payload_mass__kg_ desc \
         limit 3"

df = pd.read_sql(query, pconn)
df

,CUSTOMER,TOTAL_PAYLOAD_MASS__KG_
0,SpaceX,185220
1,Iridium Communications,67200
2,NASA (CRS),45596


### Task 4

##### Display average payload mass carried by booster version F9 v1.1


In [11]:
%sql select avg(payload_mass__kg_) as average_payload_mass__kg_ \
     from spacexdataset \
     where booster_version like 'F9 v1.1%'

 * ibm_db_sa://sbq39922:***@764264db-9824-4b7c-82df-40d1b13897c2.bs2io90l08kqb1od8lcg.databases.appdomain.cloud:32536/bludb
Done.


average_payload_mass__kg_
2534


In [46]:
query = "select booster_version, payload_mass__kg_ \
         from spacexdataset \
         where booster_version like 'F9 v1.1%'"

df = pd.read_sql(query, pconn)
df

,BOOSTER_VERSION,PAYLOAD_MASS__KG_
0,F9 v1.1 B1003,500
1,F9 v1.1,3170
2,F9 v1.1,3325
3,F9 v1.1,2296
4,F9 v1.1,1316
5,F9 v1.1,4535
6,F9 v1.1 B1011,4428
7,F9 v1.1 B1010,2216
8,F9 v1.1 B1012,2395
9,F9 v1.1 B1013,570


In [54]:
query = " select booster_version, avg(payload_mass__kg_) as average_payload_mass__kg_, count(*) as launch_count \
          from spacexdataset \
          where booster_version like 'F9 v1.1%'\
          group by booster_version "

df = pd.read_sql(query, pconn)
df

,BOOSTER_VERSION,AVERAGE_PAYLOAD_MASS__KG_,LAUNCH_COUNT
0,F9 v1.1,2928,5
1,F9 v1.1 B1003,500,1
2,F9 v1.1 B1010,2216,1
3,F9 v1.1 B1011,4428,1
4,F9 v1.1 B1012,2395,1
5,F9 v1.1 B1013,570,1
6,F9 v1.1 B1014,4159,1
7,F9 v1.1 B1015,1898,1
8,F9 v1.1 B1016,4707,1
9,F9 v1.1 B1017,553,1


In [38]:
query = "select booster_version, count(*) as launch_count \
         from spacexdataset \
         group by booster_version \
         order by launch_count desc \
         limit 3"

df = pd.read_sql(query, pconn)
df

,BOOSTER_VERSION,LAUNCH_COUNT
0,F9 v1.1,5
1,F9 B4 B1039.2,1
2,F9 B4 B1040.2,1


In [45]:
query = "select booster_version, avg(payload_mass__kg_) as average_payload_mass__kg_ \
         from spacexdataset \
         group by booster_version \
         order by average_payload_mass__kg_ desc \
         limit 10"

df = pd.read_sql(query, pconn)
df

,BOOSTER_VERSION,AVERAGE_PAYLOAD_MASS__KG_
0,F9 B5 B1048.4,15600
1,F9 B5 B1058.3,15600
2,F9 B5 B1056.4,15600
3,F9 B5 B1051.6,15600
4,F9 B5 B1051.4,15600
5,F9 B5 B1051.3,15600
6,F9 B5 B1049.7,15600
7,F9 B5 B1049.5,15600
8,F9 B5 B1049.4,15600
9,F9 B5 B1048.5,15600


### Task 5

##### List the date when the first successful landing outcome in ground pad was acheived.

*Hint:Use min function*


In [59]:
query = " select distinct(\"Landing _Outcome\") \
          from spacexdataset "

df = pd.read_sql(query, pconn)
df

,Landing _Outcome
0,Controlled (ocean)
1,Failure
2,Failure (drone ship)
3,Failure (parachute)
4,No attempt
5,Precluded (drone ship)
6,Success
7,Success (drone ship)
8,Success (ground pad)
9,Uncontrolled (ocean)


In [63]:
query = " select date, \"Time (UTC)\", \"Landing _Outcome\" \
          from spacexdataset \
          where \"Landing _Outcome\" = 'Success (ground pad)' \
          order by date asc, \"Time (UTC)\" asc \
          limit 1 \
        "

df = pd.read_sql(query, pconn)
df

,DATE,Time (UTC),Landing _Outcome
0,2015-12-22,01:29:00,Success (ground pad)


In [69]:
query = " select date, \"Time (UTC)\", \"Landing _Outcome\" \
          from spacexdataset \
          where \"Landing _Outcome\" = 'Controlled (ocean)' \
          order by date asc, \"Time (UTC)\" asc \
          limit 1 \
        "

df = pd.read_sql(query, pconn)
df

,DATE,Time (UTC),Landing _Outcome
0,2014-04-18,19:25:00,Controlled (ocean)


In [68]:
query = " select date, \"Time (UTC)\", \"Landing _Outcome\" \
          from spacexdataset \
          where \"Landing _Outcome\" = 'Success (drone ship)' \
          order by date asc, \"Time (UTC)\" asc \
          limit 1 \
        "

df = pd.read_sql(query, pconn)
df

,DATE,Time (UTC),Landing _Outcome
0,2016-04-08,20:43:00,Success (drone ship)


### Task 6

##### List the names of the boosters which have success in drone ship and have payload mass greater than 4000 but less than 6000


In [64]:
query = " select * \
          from spacexdataset \
          where \"Landing _Outcome\" = 'Success (drone ship)' and payload_mass__kg_ between 4000 and 6000 \
        "

df = pd.read_sql(query, pconn)
df

,DATE,Time (UTC),BOOSTER_VERSION,LAUNCH_SITE,PAYLOAD,PAYLOAD_MASS__KG_,ORBIT,CUSTOMER,MISSION_OUTCOME,Landing _Outcome
0,2016-05-06,05:21:00,F9 FT B1022,CCAFS LC-40,JCSAT-14,4696,GTO,SKY Perfect JSAT Group,Success,Success (drone ship)
1,2016-08-14,05:26:00,F9 FT B1026,CCAFS LC-40,JCSAT-16,4600,GTO,SKY Perfect JSAT Group,Success,Success (drone ship)
2,2017-03-30,22:27:00,F9 FT B1021.2,KSC LC-39A,SES-10,5300,GTO,SES,Success,Success (drone ship)
3,2017-10-11,22:53:00,F9 FT B1031.2,KSC LC-39A,SES-11 / EchoStar 105,5200,GTO,SES EchoStar,Success,Success (drone ship)


In [65]:
query = " select booster_version, payload_mass__kg_, \"Landing _Outcome\" as landing_outcome \
          from spacexdataset \
          where \"Landing _Outcome\" = 'Success (drone ship)' and payload_mass__kg_ between 4000 and 6000 \
        "

df = pd.read_sql(query, pconn)
df

,BOOSTER_VERSION,PAYLOAD_MASS__KG_,LANDING_OUTCOME
0,F9 FT B1022,4696,Success (drone ship)
1,F9 FT B1026,4600,Success (drone ship)
2,F9 FT B1021.2,5300,Success (drone ship)
3,F9 FT B1031.2,5200,Success (drone ship)


In [66]:
query = " select booster_version, payload_mass__kg_, \"Landing _Outcome\" as landing_outcome \
          from spacexdataset \
          where \"Landing _Outcome\" = 'Success (drone ship)' and payload_mass__kg_ < 4000 \
        "

df = pd.read_sql(query, pconn)
df

,BOOSTER_VERSION,PAYLOAD_MASS__KG_,LANDING_OUTCOME
0,F9 FT B1021.1,3136,Success (drone ship)
1,F9 FT B1023.1,3100,Success (drone ship)
2,F9 FT B1029.2,3669,Success (drone ship)
3,F9 FT B1038.1,475,Success (drone ship)
4,F9 B4 B1042.1,3500,Success (drone ship)
5,F9 B4 B1045.1,362,Success (drone ship)
6,F9 B5 B1046.1,3600,Success (drone ship)


In [67]:
query = " select booster_version, payload_mass__kg_, \"Landing _Outcome\" as landing_outcome \
          from spacexdataset \
          where \"Landing _Outcome\" = 'Success (drone ship)' and payload_mass__kg_ > 6000 \
        "

df = pd.read_sql(query, pconn)
df

,BOOSTER_VERSION,PAYLOAD_MASS__KG_,LANDING_OUTCOME
0,F9 FT B1029.1,9600,Success (drone ship)
1,F9 FT B1036.1,9600,Success (drone ship)
2,F9 B4 B1041.1,9600,Success (drone ship)


### Reference Links

*   <a href ="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Labs_Coursera_V5/labs/Lab%20-%20String%20Patterns%20-%20Sorting%20-%20Grouping/instructional-labs.md.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2021-01-01&origin=www.coursera.org">Hands-on Lab : String Patterns, Sorting and Grouping</a>

*   <a  href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Labs_Coursera_V5/labs/Lab%20-%20Built-in%20functions%20/Hands-on_Lab__Built-in_Functions.md.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2021-01-01&origin=www.coursera.org">Hands-on Lab: Built-in functions</a>

*   <a  href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Labs_Coursera_V5/labs/Lab%20-%20Sub-queries%20and%20Nested%20SELECTs%20/instructional-labs.md.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2021-01-01&origin=www.coursera.org">Hands-on Lab : Sub-queries and Nested SELECT Statements</a>

*   <a href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Module%205/DB0201EN-Week3-1-3-SQLmagic.ipynb?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2021-01-01">Hands-on Tutorial: Accessing Databases with SQL magic</a>

*   <a href= "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Module%205/DB0201EN-Week3-1-4-Analyzing.ipynb?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2021-01-01">Hands-on Lab: Analyzing a real World Data Set</a>


## Author(s)

<h4> Lakshmi Holla </h4>


## Other Contributors

<h4> Rav Ahuja </h4>


## Change log

| Date       | Version | Changed by    | Change Description        |
| ---------- | ------- | ------------- | ------------------------- |
| 2021-10-12 | 0.4     | Lakshmi Holla | Changed markdown          |
| 2021-08-24 | 0.3     | Lakshmi Holla | Added library update      |
| 2021-07-09 | 0.2     | Lakshmi Holla | Changes made in magic sql |
| 2021-05-20 | 0.1     | Lakshmi Holla | Created Initial Version   |


## <h3 align="center"> © IBM Corporation 2021. All rights reserved. <h3/>
